
"""

Treatement of Categorical Variables:

One Hot Encoding
- Distance Group

Frequency Enconding 
- Track
- Trainer

NEXT STEPS
Entity Embeddings for Neural Networks going forward

"""



# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

df.ft_raw = pd.read_csv(f'{fileloc}/raw_base_2022-12-03.csv', parse_dates = True)

print(df.ft_raw.shape)
print(df.ft_raw.columns.values.tolist())


(588017, 33)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos']


### B) Create Variable for Categories

In [8]:

df_base_cats = df.ft_raw.copy()


In [9]:

# Sort for Easy QC
df_base_cats.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


In [10]:

df_base_cats[df_base_cats['@id_DOG'].isin([108390981])]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,y_margin,y_pos
0,108390981,4,DOLLY DOES IT,5.0,5,30.4,$11.00,NaN,6.00,0.14,0,0,B3,NaN,26.43,80.0,626271182,92892,M Elphinstone,626271182,10,GREAT NORTHERN,03:14PM,452m,Mixed 4/5,Devonport,05 Jan 21,2021-01-05,11.0,8,4,6.00,4
1,108390981,5,DOLLY DOES IT,4.0,4,30.2,$7.00,NaN,6.00,1.14,0,0,NaN,NaN,26.40,0.0,629797242,92892,M Elphinstone,629797242,10,GREAT NORTHERN,03:09PM,452m,Grade 5,Devonport,12 Jan 21,2021-01-12,7.0,7,5,6.00,5
2,108390981,1,DOLLY DOES IT,5.0,5,30.2,$7.00,NaN,5.25,NaN,0,0,NaN,8.1,26.20,1890.0,629797513,92892,M Elphinstone,629797513,4,SUREPICK,01:27PM,452m,Grade 5,Devonport,19 Jan 21,2021-01-19,7.0,7,1,5.25,1
3,108390981,7,DOLLY DOES IT,1.0,1,30.0,$7.00,NaN,10.50,2.00,0,0,NaN,NaN,26.48,0.0,638810808,92892,M Elphinstone,638810808,8,LADBROKES,02:22PM,452m,Mixed 4/5 Heat,Devonport,16 Feb 21,2021-02-16,7.0,8,7,10.50,7
4,108390981,8,DOLLY DOES IT,8.0,8,29.9,$91.00,NaN,18.50,3.57,0,0,SO,NaN,26.91,0.0,643554706,92892,M Elphinstone,643554706,8,NEW RECRUIT 2012,02:22PM,452m,Grade 4,Devonport,02 Mar 21,2021-03-02,91.0,8,8,18.50,8


In [11]:

# ADDITIONAL VARIABLES
doglevel = ['@id_DOG']

# Distance and Speed in each race
df_base_cats['x_dist'] = df_base_cats['Distance'].apply(lambda x : x.replace('m','')).astype(int)
#df_base_cats['speed'] = (df_base_cats['x_dist'] / df_base_cats['y_runtime'])*1000

# GROUPED X_DIST 
df_base_cats['x_dist_gp'] = df_base_cats['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)

# Flag for getting a total race count
df_base_cats['f_1'] = 1

# Total Races
df_base_cats['x_races_tot'] = df_base_cats.groupby(doglevel).cumcount()


In [12]:

print(df_base_cats.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'x_dist_gp', 'f_1', 'x_races_tot']


In [13]:

print(df_base_cats.shape)


(588017, 37)


In [14]:

ft_w_dummies = pd.get_dummies(df_base_cats['x_dist_gp']).add_prefix('dist')

print(ft_w_dummies.shape)
print(ft_w_dummies.columns.values.tolist())
print()

df_base_cats_w_dummies = pd.concat([df_base_cats, ft_w_dummies], axis = 1)

print(df_base_cats_w_dummies.shape)
print(df_base_cats_w_dummies.columns.values.tolist())
print()


(588017, 8)
['dist-999', 'dist300', 'dist350', 'dist400', 'dist450', 'dist500', 'dist600', 'dist700']

(588017, 45)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'x_dist_gp', 'f_1', 'x_races_tot', 'dist-999', 'dist300', 'dist350', 'dist400', 'dist450', 'dist500', 'dist600', 'dist700']



In [15]:

pd.crosstab( df_base_cats_w_dummies['dist300'], df_base_cats_w_dummies['x_dist_gp'])


x_dist_gp,-999,300,350,400,450,500,600,700
dist300,,,,,,,,
0,6,0,72466,177559,104059,119365,15595,5829
1,0,93138,0,0,0,0,0,0



### Exporting the Dataset


In [16]:

keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Distance','x_dist_gp', 'dist300', 'dist350', 'dist400', 'dist450', 'dist500', 'dist600', 'dist700']
#'Event_Dt', 

cols_for_drop = []

todaydt = datetime.now().date()
exp_fname = f'dataprep_categorical_{todaydt}.csv'

exp_ds = df_base_cats_w_dummies[keep_cols_for_export]


In [17]:

print(exp_ds.shape)


(588017, 13)


In [18]:

print(exp_ds.columns.values.tolist())


['@id_DOG', 'DogName', '@id_RACE', 'RaceName', 'Distance', 'x_dist_gp', 'dist300', 'dist350', 'dist400', 'dist450', 'dist500', 'dist600', 'dist700']


In [19]:

exp_ds.head()


,@id_DOG,DogName,@id_RACE,RaceName,Distance,x_dist_gp,dist300,dist350,dist400,dist450,dist500,dist600,dist700
0,108390981,DOLLY DOES IT,626271182,GREAT NORTHERN,452m,450,0,0,0,1,0,0,0
1,108390981,DOLLY DOES IT,629797242,GREAT NORTHERN,452m,450,0,0,0,1,0,0,0
2,108390981,DOLLY DOES IT,629797513,SUREPICK,452m,450,0,0,0,1,0,0,0
3,108390981,DOLLY DOES IT,638810808,LADBROKES,452m,450,0,0,0,1,0,0,0
4,108390981,DOLLY DOES IT,643554706,NEW RECRUIT 2012,452m,450,0,0,0,1,0,0,0


In [20]:

exp_ds.describe()


,@id_DOG,@id_RACE,x_dist_gp,dist300,dist350,dist400,dist450,dist500,dist600,dist700
count,5.880170e+05,5.880170e+05,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000
mean,4.619574e+08,7.386968e+08,415.410534,0.158393,0.123238,0.301962,0.176966,0.202996,0.026521,0.009913
std,1.145309e+08,6.357945e+07,78.101987,0.365110,0.328710,0.459109,0.381640,0.402230,0.160680,0.099069
min,1.083910e+08,6.238619e+08,-999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.005005e+08,6.854128e+08,350.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.569441e+08,7.402821e+08,400.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.405822e+08,7.922435e+08,450.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,2.145360e+09,8.483914e+08,700.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [22]:

print(datetime.now())
print()
print('THE END')


2022-12-03 04:42:50.466241

THE END
